In [1]:
#סעיף א
import pandas as pd

#המרה לקובץ טקסט והוספת כותרות לעמודות
def add_columns(file_path, txt_path):
    """פורמט מבוקש עם כותרות TXT לקובץ Excel המרת קובץ"""
    df = pd.read_excel(file_path, header=None, names=['Raw'])
    df[['Timestamp', 'Error']] = df['Raw'].str.split(', ', expand=True)
    df['Timestamp'] = df['Timestamp'].str.replace('Timestamp: ', '', regex=False)
    df['Error'] = df['Error'].str.replace('Error: ', '', regex=False)
    df = df[['Timestamp', 'Error']] 
    df.to_csv(txt_path, index=False, sep=',', header=True)  
    print(f"הקובץ נשמר בהצלחה {txt_path}")

add_columns('logs.txt.xlsx', 'logs.txt')

הקובץ נשמר בהצלחה logs.txt


In [2]:
from collections import Counter

def count_error_frequencies(file_path, chunk_size=10000):
    """ספירת השגיאות בכל chunk"""
    error_counter = Counter()  
    for chunk in pd.read_csv(file_path, header=0, names=['Timestamp', 'Error'], chunksize=chunk_size):
        error_counter.update(chunk['Error'])  
    return error_counter

def top_n_error_codes(file_path, n, chunk_size=10000):
    """מציאת N קודי השגיאה השכיחים ביותר"""
    error_counter = count_error_frequencies(file_path, chunk_size)
    return error_counter.most_common(n)  

n = 2  
top_errors = top_n_error_codes('logs.txt', n)
print(f"ה-{n} קודי השגיאה השכיחים ביותר: {top_errors}")


ה-2 קודי השגיאה השכיחים ביותר: [('WARN_101', 200098), ('ERR_404', 200094)]


In [3]:
#בדיקה שזה אכן נכון

import pandas as pd
def load_data(file_path='logs.txt'):
    df = pd.read_csv(file_path)
    return df
df=load_data()
print(df['Error'].value_counts())

WARN_101    200098
ERR_404     200094
ERR_400     200069
INFO_200    199931
ERR_500     199808
Name: Error, dtype: int64


<div style="direction: rtl; text-align: right;">
### סיבוכיות זמן

### חלוקה ל-2 מקרים:

1.
### זמן ריצה ממוצע:
   - **קריאת שורה מקובץ, חיפוש והוספה ב-Counter.update()**:
     - פעולת החיפוש וההוספה במילון פנימי של ה-Counter (המבוסס על hashing table) מתבצעת בזמן O(1) בממוצע.
     - לכן, עבור כל פריט בקובץ הלוגים, עדכון ספירת השגיאה הוא פעולה של O(1) בממוצע.
     - לכן, הזמן הכולל של עדכון ספירת השגיאות הוא O(M) כש-M הוא מספר השורות בקובץ.
   - **מיון השגיאות השכיחות ביותר**:
     - לאחר עדכון הספירות, צריך למיין את השגיאות השכיחות ביותר.
     - זמן הריצה של מיון הוא O(K log K), כש-K הוא מספר השגיאות השונות (מפתחות המילון).
     - K יכול להגיע לסדר גודל של M, אם כל שורה מכילה שגיאה ייחודית ולכן זמן הריצה הוא O(M log M).

   ### סיבוכיות זמן ממוצעת:
   - **סיבוכיות כוללת**: O(M + M log M) = O(M log M).

2.
 ### זמן ריצה במקרה הגרוע:
   - במקרה הגרוע, קונפליקטים ב-hashing עשויים לגרום לפעולה איטית יותר.
   - במקרה כזה, זמן הריצה עשוי להגיע ל-O(M) לכל חיפוש או עדכון של שורה.
   - לכן, הזמן הכולל של עדכון ספירת השגיאות הוא O(M * M).
   - **מיון השגיאות השכיחות ביותר**: שלב המיון נשאר זהה, O(M log M).

   ### סיבוכיות זמן במקרה הגרוע:
   - **סיבוכיות כוללת**: O(M^2 + M log M) = O(M^2).

## סיכום:
- **ממוצע**: סיבוכיות הזמן היא O(M + M log M).
- **מקרה גרוע**: סיבוכיות הזמן היא O(M * M + M log M).

## סיבוכיות מקום:
- **O(M)** במקרה הגרוע.
- **הסבר**: ספירת השגיאות מתבצעת באמצעות Counter, שהוא בעצם מילון (Dictionary). כל "קוד שגיאה" שנמצא יתווסף כ-מפתח במילון, ומספר השגיאות שהוא נמצא בהן יתעדכן כערך.
- **במקרה הגרוע**: כל שורה בקובץ יכולה להכיל קוד שגיאה שונה, כלומר מספר השגיאות השונות (ה-keys במילון) עשוי להיות שווה למספר השורות M.

</div>

In [4]:
#סעיף ב
#חלק 1 -ביצוע בדיקות
def is_parquet_file(file_path):
    return file_path.lower().endswith('.parquet')

def load_data(file_path):
    if is_parquet_file(file_path):
        df=pd.read_parquet(file_path)
        return df
    else:
        df=pd.read_excel(file_path)
        return df
df=load_data('time_series.parquet')
print(df.columns)


Index(['timestamp', 'value'], dtype='object')


In [5]:
#חקירה ראשונית של הנתונים
def initial_exploration(df):
    print("סטטיסטיקות בסיסיות:")
    print(df.describe())
    print("\nאורך הדטה:", len(df))
    print("\nשורות ראשונות:")
    print(df.head())
    print("\nסוגי נתונים:")
    print(df.dtypes)
    print("\nערכים חסרים:")
    print(df.isnull().sum())
    print("\nשמות עמודות:")
    print(df.columns)
initial_exploration(df)

סטטיסטיקות בסיסיות:
               value
count  995566.000000
mean       49.962300
std        26.675291
min         0.000000
25%        29.200000
50%        49.962300
75%        70.700000
max       100.000000

אורך הדטה: 995566

שורות ראשונות:
             timestamp  value
0  2025-06-28 12:00:52   18.5
1  2025-06-01 04:17:23   46.3
2  2025-06-10 17:02:57   76.0
3  2025-06-23 05:23:22   56.4
4  2025-06-05 07:20:08   67.9

סוגי נתונים:
timestamp     object
value        float64
dtype: object

ערכים חסרים:
timestamp    0
value        0
dtype: int64

שמות עמודות:
Index(['timestamp', 'value'], dtype='object')


In [ ]:
# מסקנות מהחקירה:
# העמודה timestamp בפורמט תקין ואין לה ערכים חסרים
# יש לבצע בעמודה Value טיפול בערכים חסרים
# עמודה Value 
# אינה מטיפוס מספר ולכן המסקנה היא שנכנסו נתונים שגויים
# (top 2025-06-26 03:00:40  not_a_number)
# ויש להסירם

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# פונקציה לבדוק את סוגי הנתונים בעמודות
def check_data_types(df):
    print("\nסוגי נתונים:")
    print(df.dtypes)

# פונקציה לבדוק ולתקן פורמט תאריך בעמודה 'timestamp'
def check_and_fix_timestamp(df):
    if pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        print("\nהעמודה 'timestamp' בפורמט תאריך תקני.")
    else:
        print("\nהעמודה 'timestamp' אינה בפורמט תאריך תקני. מבצע המרה...")
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        print("לא תקינים הומרו ל-NaT.")
    return df

# פונקציה לבדוק ערכים חסרים בעמודות
def check_missing_values(df):
    missing_values = df.isnull().sum()
    print("\nערכים חסרים:")
    print(missing_values)
    return missing_values

# פונקציה לבדוק כפילויות ולמחוק אותן
def check_and_remove_duplicates(df):
    duplicates = df.duplicated().sum()
    print(f"\nמספר כפילויות: {duplicates}")
    df = df.drop_duplicates()
    return df


#בודקת את הערכים הלא-מספריים בעמודה value
def check_non_numeric_values(df):
    non_numeric_values = df[~df['value'].apply(pd.to_numeric, errors='coerce').notna()]
    unique_non_numeric_values = non_numeric_values['value'].unique()
    print("ערכים לא מספריים ייחודיים בעמודה 'value':")
    print(unique_non_numeric_values)
    
#הפונקציה מחזירה את ה-DataFrame המעודכן, שבו הערכים הלא-מספריים בעמודה value הומרו ל-NaN
def handle_non_numeric_in_value(df):
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    return df

# פונקציה למלא ערכים חסרים בעמודות עם ערכים לא מספריים או מספריים
def fill_missing_values(df):
    for column in df.columns:
        if df[column].dtype == 'object': 
            df[column] = df[column].fillna(df[column].mode()[0])  
        else:  # אם מדובר בעמודה מספרית
            if pd.api.types.is_numeric_dtype(df[column]): 
                if df[column].skew() > 1: 
                    df[column] = df[column].fillna(df[column].median())  
                else:
                    df[column] = df[column].fillna(df[column].mean())  
    return df

# פונקציה ראשית שמבצעת את כל הבדיקות והטיפולים
def process_data(df):
    df = check_and_fix_timestamp(df)
    check_missing_values(df)
    df = check_and_remove_duplicates(df)
    check_non_numeric_values(df)
    df = handle_non_numeric_in_value(df)
    df = fill_missing_values(df)
    return df
 
df = process_data(df)  
df.to_csv("processed_data.csv", index=False, encoding="utf-8")


העמודה 'timestamp' אינה בפורמט תאריך תקני. מבצע המרה...
לא תקינים הומרו ל-NaT.

ערכים חסרים:
timestamp    0
value        0
dtype: int64

מספר כפילויות: 0
ערכים לא מספריים ייחודיים בעמודה 'value':
[]


In [7]:
#בדיקת הקובץ החדש אם הכל תקין עכשיו
df=pd.read_csv('processed_data.csv')
initial_exploration(df)

סטטיסטיקות בסיסיות:
               value
count  995566.000000
mean       49.962300
std        26.675291
min         0.000000
25%        29.200000
50%        49.962300
75%        70.700000
max       100.000000

אורך הדטה: 995566

שורות ראשונות:
             timestamp  value
0  2025-06-28 12:00:52   18.5
1  2025-06-01 04:17:23   46.3
2  2025-06-10 17:02:57   76.0
3  2025-06-23 05:23:22   56.4
4  2025-06-05 07:20:08   67.9

סוגי נתונים:
timestamp     object
value        float64
dtype: object

ערכים חסרים:
timestamp    0
value        0
dtype: int64

שמות עמודות:
Index(['timestamp', 'value'], dtype='object')


In [8]:
import os
import pandas as pd

chunk_size = 10000
chunks = pd.read_csv('processed_data.csv', chunksize=chunk_size)
all_data = pd.DataFrame()

for chunk in chunks:
    chunk['timestamp'] = pd.to_datetime(chunk['timestamp'])
    all_data = pd.concat([all_data, chunk], ignore_index=True)
    
all_data['date'] = all_data['timestamp'].dt.date
df_daily = all_data.groupby('date')

output_dir = 'split_files'
os.makedirs(output_dir, exist_ok=True)

for date, group in df_daily:
    print(f"נתונים ליום: {date}")
    group.to_csv(f"{output_dir}/time_series_{date}.csv", index=False)


נתונים ליום: 2025-06-01
נתונים ליום: 2025-06-02
נתונים ליום: 2025-06-03
נתונים ליום: 2025-06-04
נתונים ליום: 2025-06-05
נתונים ליום: 2025-06-06
נתונים ליום: 2025-06-07
נתונים ליום: 2025-06-08
נתונים ליום: 2025-06-09
נתונים ליום: 2025-06-10
נתונים ליום: 2025-06-11
נתונים ליום: 2025-06-12
נתונים ליום: 2025-06-13
נתונים ליום: 2025-06-14
נתונים ליום: 2025-06-15
נתונים ליום: 2025-06-16
נתונים ליום: 2025-06-17
נתונים ליום: 2025-06-18
נתונים ליום: 2025-06-19
נתונים ליום: 2025-06-20
נתונים ליום: 2025-06-21
נתונים ליום: 2025-06-22
נתונים ליום: 2025-06-23
נתונים ליום: 2025-06-24
נתונים ליום: 2025-06-25
נתונים ליום: 2025-06-26
נתונים ליום: 2025-06-27
נתונים ליום: 2025-06-28
נתונים ליום: 2025-06-29
נתונים ליום: 2025-06-30


In [9]:
import pandas as pd
import glob
import os

def process_daily_files(input_dir='split_files/', output_path='hourly_averages.csv'):
    files = glob.glob(os.path.join(input_dir, 'time_series*.csv'))
    all_hourly_averages = []
    
    for file_path in files:
        print(f"קורא את הקובץ: {file_path}")
        df = pd.read_csv(file_path)
        hourly_avg = calculate_hourly_avg(df)
        all_hourly_averages.append(hourly_avg)
        
    final_df = pd.concat(all_hourly_averages, ignore_index=True)
    final_df.to_csv(output_path, index=False)
    print(f"התוצאה נשמרה ב: {output_path}")

def calculate_hourly_avg(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = df['timestamp'].dt.floor('H')  
    hourly_avg = df.groupby('hour')['value'].mean().reset_index()
    hourly_avg.columns = ['time_start', 'average']
    hourly_avg['time_start'] = hourly_avg['time_start'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return hourly_avg

process_daily_files()

קורא את הקובץ: split_files\time_series_2025-06-01.csv
קורא את הקובץ: split_files\time_series_2025-06-02.csv
קורא את הקובץ: split_files\time_series_2025-06-03.csv
קורא את הקובץ: split_files\time_series_2025-06-04.csv
קורא את הקובץ: split_files\time_series_2025-06-05.csv
קורא את הקובץ: split_files\time_series_2025-06-06.csv
קורא את הקובץ: split_files\time_series_2025-06-07.csv
קורא את הקובץ: split_files\time_series_2025-06-08.csv
קורא את הקובץ: split_files\time_series_2025-06-09.csv
קורא את הקובץ: split_files\time_series_2025-06-10.csv
קורא את הקובץ: split_files\time_series_2025-06-11.csv
קורא את הקובץ: split_files\time_series_2025-06-12.csv
קורא את הקובץ: split_files\time_series_2025-06-13.csv
קורא את הקובץ: split_files\time_series_2025-06-14.csv
קורא את הקובץ: split_files\time_series_2025-06-15.csv
קורא את הקובץ: split_files\time_series_2025-06-16.csv
קורא את הקובץ: split_files\time_series_2025-06-17.csv
קורא את הקובץ: split_files\time_series_2025-06-18.csv
קורא את הקובץ: split_files\t

In [10]:
#בדיקה
df_new = pd.read_csv("hourly_averages.csv")
print(df_new.head())  
print(df_new.info())  

            time_start    average
0  2025-06-01 00:00:00  50.486972
1  2025-06-01 01:00:00  49.942680
2  2025-06-01 02:00:00  49.536653
3  2025-06-01 03:00:00  50.151774
4  2025-06-01 04:00:00  48.803489
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   time_start  720 non-null    object 
 1   average     720 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.4+ KB
None


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif;">
  <h1>פרק 3</h1>
  <hr>
  <h2>--- תכנון ---</h2>

  <h3>מחלקה RealTimeHourlyAverages ובה:</h3>
  <h4>א. `hourly_averages` - מבנה נתונים:</h4>
  <b>הסבר:</b> נשתמש במילון שבו כל שעה תישמר עם ממוצע נתוניה:
  <ul>
    <li><b>מפתח:</b> שעה</li>
    <li><b>ערך:</b> אובייקט<b>HourlyStats</b> שיאפשר לעדכן את הממוצע בזמן אמת שמכיל :
      <ul>
        <li>`sum` – סכום כל הערכים שהתקבלו עד כה עבור השעה.</li>
        <li>`count` – מספר הערכים שהתקבלו עד כה עבור השעה.</li>
        <li>`add_value()`</li>
        <li>`get_average()`</li>
      </ul>
    </li>
  </ul>

  <h4>ב. `update_Hourly_Average()`</h4>

  <h4>ג. `get_Hourly_Average()`</h4>

  <hr>
  <h2>--- תהליך העבודה ---</h2>
  <ul>
    <li><b>קבלת נתון חדש:</b> כל פעם שתקבל נתון חדש (בזמן אמת), תזין אותו לשעה המתאימה.</li>
    <li><b>עדכון הממוצע:</b> עבור כל שעה, תעדכן את הסכום והכמות, ואז תחזיר את הממוצע החדש.</li>
    <li><b>חישוב הממוצע:</b> הממוצע יחושב בכל פעם שיידרש, פשוט על ידי חלוקה של הסכום בכמות.</li>
  </ul>

  <hr>
  <h2>--- תכנון פונקציות ---</h2>

  <h3>HourlyStats:</h3>
  <ul>
    <li>`add_value(value)`: מוסיף ערך חדש ומעדכן את הסכום והכמות.</li>
    <li>`get_average()`: מחשב את הממוצע על ידי חלוקה של הסכום בכמות.</li>
  </ul>

  <h3>RealTimeHourlyAverages:</h3>
  <ul>
    <li>`update_Hourly_Average(hour, value)`: מעדכן את הממוצע של השעה על ידי הוספת הערך החדש לאובייקט המתאים בשעה המתאימה.</li>
    <li>`get_Hourly_Average(hour)`: מחזיר את הממוצע של השעה.</li>
  </ul>
</div>

In [28]:
class Stats:
    def __init__(self):
        self.count = 0
        self.total = 0.0

    def add(self, value):
        self.total += value
        self.count += 1

    def average(self):
        return self.total / self.count if self.count > 0 else 0

class HourlyAverages:
    def __init__(self):
        self.averages = {}

    def process(self, hour, value):
        if hour not in self.averages:
            self.averages[hour] = Stats()
        self.averages[hour].add(value)

    def get_average(self, hour):
        return self.averages.get(hour, Stats()).average()

average_calculator = HourlyAverages()

average_calculator.process(14, 10.5)
average_calculator.process(14, 12.0)
average_calculator.process(15, 8.5)

print("ממוצע לשעה 14:", average_calculator.get_average(14))
print("ממוצע לשעה 15:", average_calculator.get_average(15))


ממוצע לשעה 14: 11.25
ממוצע לשעה 15: 8.5


<div style="direction: rtl; text-align: right; font-family: Arial, sans-serif;">
    <div style="text-align: right;">קובץ Parquet</div>
    <p style="text-align: right;"> לקובץ parquet קיימים יתרונות רבים כאשר יש טיפול מקדים בנתונים.</p>
    <p style="text-align: right;">קובץ Parquet עוזר בזיהוי בעיות בנתונים מראש, דבר שיכול לחסוך זמן ומשאבים בתהליך עיבוד הנתונים.</p>
    <hr style="width: 50%; margin-right: auto; margin-left: 0;">
    <p style="text-align: right;"><b>לדוגמה:</b></p>
    <p style="text-align: right;">אם יש בעיות עם נתונים לא נכונים בעמודה, למשל מילה במקום מספר, השמירה לקובץ לא תעבור בהצלחה.</p>
    <p style="text-align: right;">זה עוזר לזהות בעיות בנתונים לפני השמירה, ומונע טעויות שעשויות להתרחש בשלב מאוחר יותר בתהליך.</p>
    <p style="text-align: right;">ולכן אין צורך לבצע בדיקות מסוג זה על קובץ Parquet בטעינתו, כי כל הבעיות כבר מזוהות מראש.</p>
    <hr style="width: 50%; margin-right: auto; margin-left: 0;">
    <p style="text-align: right;">יתרון נוסף וחשוב של קובץ Parquet הוא שבגלל שהוא מבוסס על מבנה נתונים קולומנרי, הוא אופטימלי למקרים של קריאה וכתיבה של נתונים באופן סלקטיבי, כלומר, ניתן לקרוא עמודות מסוימות בלבד מבלי לטעון את כל הקובץ.</p>
    <p style="text-align: right;">זה מאפשר שיפור משמעותי בזמני טעינה של הנתונים, במיוחד בקבצים גדולים מאוד.</p>
    <p style="text-align: right;">בנוסף, קובץ Parquet משתמש בשיטות דחיסה מתקדמות, שמקטינות את הגודל של הקובץ ומייעלות את השימוש בזיכרון ובזמן המעבד, מה שגורם להאצת תהליך הקריאה והכתיבה של נתונים.</p>
    <p style="text-align: right;">כמו כן, מכיוון שקובץ Parquet הוא פורמט פתוח, הוא נתמך על ידי הרבה מערכות ו- frameworks שונים כמו Apache Spark, Apache Hive ו-Pandas, מה שמבטיח תאימות גבוהה עם כלים רבים בתחום ניתוח הנתונים.</p>
    <p style="text-align: right;">יתרון זה מאוד מועיל במקרים שבהם רוצים לוודא שהנתונים נשמרים בצורה נכונה ויעילה, לפני שממשיכים את העבודה עליהם או מבצעים אנליזות נוספות.</p>
    <br>
    <p style="text-align: right;">קובץ Parquet יעבוד על הקוד. השינוי היחיד שצריך לעשות הוא התאמה לטעינת קובץ מסוג Parquet.</p>
    <pre style="text-align: left; direction: ltr; background-color: #f4f4f4; padding: 10px; border: 1px solid #ddd; border-radius: 4px;">
        <code>
def is_parquet_file(file_path):
    return file_path.lower().endswith('.parquet')

if is_parquet_file('time_series.parquet'):
    pd.read_parquet('time_series.parquet', columns=columns)
        </code>
    </pre>
</div>